In [ ]:
import importlib, os, sys
import numpy as np
from scipy.interpolate import interp1d as interp1d
import datetime

# Go one level up and add it to sys.path such that DarkAges can be found.
darkpath = os.path.dirname(os.path.abspath(os.curdir))
sys.path.insert(1,darkpath)
sys.path.insert(1,'/mnt/c/Linux/ExoCLASS_DMwEL/DarkAgesModule')
sys.path.insert(1,'/mnt/c/Linux/ExoCLASS_DMwEL/DarkAgesModule/DarkAges')
#C:\LinuxExoCLASS_DMwEL/DarkAgesModule/bin
#from DarkAges.evaporator import PBH_F_of_M as F_of_M
#from DarkAges.evaporator import get_temperature_from_mass, get_mass_from_temperature

%matplotlib inline
import matplotlib.pyplot as plt
from math import pi
import matplotlib
from matplotlib import ticker
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

plt.rc('text', usetex=True)
#plt.rc('text.latex', unicode=True)
plt.rc('font', family='sans serif')

pgf_with_custom_preamble = {
	"font.family": "sans serif",
	"text.usetex": True,
	"text.latex.unicode": True,
	"pgf.rcfonts": False
}

channelIndex = ['Heat','H-Ion']


In [ ]:
# import classy module
from classy import Class
from DarkAges import transfer_functions
from DarkAges.model import dmwel_model #as model
from DarkAges.model import create_f_dmwel
from DarkAges.transfer import transfer_combine
from DarkAges.__init__ import channel_dict
from DarkAges.common import finalize
# as can be seen from definition __init__.channel_dict, defintion of function common.finalize
    # and use of finalize in recipes.py, the following from channel_dict are the indices in f_function
#'H-Ion': 0,
#	'He-Ion': 1,
#	'Ly-A': 2,
#	'Heat': 3,
#	'LowE': 4
        
total_transfer = transfer_combine(*transfer_functions)

In [ ]:
###### create instance of the class "Class"
PDGPl18 = Class()
# pass input parameters - the omegas are from PDG18 and the rest from Planck 2018
PDGPl18.set({'omega_b':0.02242,'omega_cdm':0.11933,'h':0.6766,'A_s':2.105e-9,'n_s':0.9665,'tau_reio':0.0561})
PDGPl18.set({'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':3.0})
# run class
PDGPl18.compute()

In [ ]:
# get all C_l output
cls = PDGPl18.lensed_cl(2500)
# To check the format of cls
cls.keys() #need to change original 'dictkeys' to 'keys' for python3 compatibility

In [ ]:
create_f_dmwel()

In [ ]:
rs = np.geomspace(11.918400, 2747.970000, num=63)  #np.arange(1.,2501.,1.)
xepre = PDGPl18.ionization_fraction
xe = np.ones_like(rs)
for idx,redshift in enumerate(rs):
    xe[idx] = xepre(redshift)

In [ ]:
# plot xe
plt.figure(1)
plt.xscale('log');plt.yscale('log');plt.xlim(300,1200)
plt.xlabel(r'$z$')
plt.ylabel(r'$x_\mathrm{e}(z)$')
plt.plot(rs,xe,'g-')

In [ ]:
GSVIxe=[0,0.0001,0.0003,0.0005,0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,0.8,0.9,0.99,1.1,1.2]
GSVIchi_ionH=[0.350798,0.350798,0.349058,0.345508,0.341822,0.327298,0.316798,0.301893,0.255925,0.228453,0.175739,0.083885,0.043901,0.013518,0.006406,0.0017,0,0]
GSVIchi_heat=[0.1518,0.15188,0.174825,0.18852,0.210027,0.258912,0.289871,0.338316,0.458621,0.531628,0.654816,0.849031,0.923644,0.975679,0.987026,0.995299,1,1]

In [ ]:
chi_ionH_fn=interp1d(GSVIxe,GSVIchi_ionH)
chi_ionH = chi_ionH_fn(xe)
chi_heat_fn=interp1d(GSVIxe,GSVIchi_heat)
chi_heat = chi_heat_fn(xe)

In [ ]:
# plot chi_ionH
fig = plt.figure(1)
ax = plt.subplot(111)
plt.xscale('log');plt.yscale('linear');plt.xlim(300,1200);
ax.set_ylim(-0.05,1.05)
plt.xlabel(r'$1+z$')
ax.plot(rs,chi_ionH,'b--',label='$\chi_{\mathrm{H\!\!-\!\!ion}}$')
ax.plot(rs,chi_heat,'r--',label='$\chi_\mathrm{heat}$')
ax.legend(loc='best')
plt.savefig('GSVI_chi.pdf')

In [ ]:
def data_header(filename,this_file,comment,labels):
    now = datetime.datetime.now()
    
    fname = open(filename,'w+')
    fname.write('(@@\n{{"%s/%s","%s"},"%s"}\n@@)\n\n(<<\n{'%(os.path.abspath(os.curdir),this_file,now.strftime("%Y-%m-%d--%H:%M:%S"),comment))
    
    for i in range(len(labels)-1):
        fname.write('\"%s\",'%labels[i])
    fname.write('\"%s\"}\n>>)\n\n(::\n{\n'%labels[-1])
    
    return(fname)

def arraywrite_2f(fname,a):
    fname.write('{')
    for i in range(len(a)-1):
        fname.write('{') 
        for j in range(len(a[i])-1):
            fname.write('%.10f,'%a[i,j])
        fname.write('%.10f},\n'%a[i,-1])
    fname.write('{')
    for j in range(len(a[-1])-1):
        fname.write('%.10f,'%a[-1,j])
    fname.write('%.10f}}'%a[-1,-1])
    
def arraywrite_3f(fname,a):
    fname.write('\n{\n')
    for i in range(len(a)-1):
        arraywrite_2f(fname,a[i])
        fname.write(',')
    arraywrite_2f(fname,a[-1])
    fname.write('\n}\n')
    
def data_footer(fname):
    fname.write('\n}\n::)')
    fname.close()

For some reason when running the whole sheet, it often hangs on the next cell.  Alternative is to go to the cell below "Run All Above" and then manually run the cells below.  Googling suggests this is a Jupyter bug.

In [ ]:
GSVI_chi = np.zeros((3,len(rs)))
GSVI_chi[0] = rs
GSVI_chi[1] = chi_heat
GSVI_chi[2] = chi_ionH

In [ ]:
log_energy_list = np.linspace(-2.,4.,7)

dp_h = np.array([[0.0001,1.],[0.0001,2.],[0.001,10.],[0.001,20.],[0.01,100.],[0.01,200.]])
dp_4 = np.transpose(np.array((1e-4 * np.ones((7)),10.**log_energy_list)))
dp_3 = np.transpose(np.array((1e-3 * np.ones((7)),10.**log_energy_list)))
dp_2 = np.transpose(np.array((1e-2 * np.ones((7)),10.**log_energy_list)))

dmwel_parameter_set = np.concatenate((dp_h,dp_4,dp_3,dp_2))


fname = data_header('dmwel_chi_and_fc.data','dmwel_chi_and_fc.ipynb','chi, dmwel parameters, and fc',['chi','parameters','fc'])
fname.write('\n')
arraywrite_2f(fname,GSVI_chi)
fname.write('\n,\n')

arraywrite_2f(fname,dmwel_parameter_set)
fname.write('\n,\n')

for j,dmwel_para in enumerate(dmwel_parameter_set):
    #print(j)
    dmwel_alpha1 = dmwel_para[0]
    dmwel_e1 = dmwel_para[1]
    print('paras = {},{}'.format(dmwel_alpha1,dmwel_e1))
    f_functions = np.zeros((1))
    max_j = min(1000,np.floor(1000*np.sqrt(0.001/dmwel_alpha1)).astype(int))
    print('max_j = {}'.format(max_j))
    for i,channel in enumerate(channelIndex):
        idx = channel_dict[channel]
        #print(idx)
        temporary = dmwel_model(dmwel_alpha1,dmwel_e1,max_j,redshift=rs).calc_f(transfer_functions[idx],E_integration_scheme = 'dmwel')
        if f_functions.shape == (1,):
            redshift = temporary[0]
            f_functions = np.zeros((len(channelIndex),len(redshift)))
        f_functions[i,:] = temporary[-1]
    #print(redshift.shape)
    #print(f_functions.shape)
    
    if j == 0:
        dmwel_fc = np.zeros((len(dmwel_parameter_set),3,len(redshift)-1))
    dmwel_fc[j,0,:] = redshift[:-1]
    dmwel_fc[j,1,:] = f_functions[0,:-1]
    dmwel_fc[j,2,:] = f_functions[1,:-1]
#print(dmwel_fc)
arraywrite_3f(fname,dmwel_fc)
#fname.write('\n}\n')

data_footer(fname)

In [ ]:
print('Completed')

In [ ]:
# optional: clear content of PDGPl18 (to reuse it for another model)
PDGPl18.struct_cleanup()
# optional: reset parameters to default
PDGPl18.empty()

In [ ]:
dmwel_model(dmwel_alpha1,dmwel_e1,max_j,redshift=rs).dmwel_f(0.001,10.,1)